In [5]:
import os
import random
import shutil
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import albumentations as A
from tqdm.auto import tqdm

/home/yesman/tfvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
print("--- Етап 1: Конфігурація YOLO ---")

BASE_DIR = 'learning'
# Шляхи до шрифтів
FONT_DIR = os.path.join(BASE_DIR, 'fonts')
VALID_FONT_DIR = os.path.join(BASE_DIR, 'fonts_valid')
YOLO_DIR = os.path.join(BASE_DIR, 'data_number')

YOLO_TRAIN_IMG = os.path.join(YOLO_DIR, 'train', 'images')
YOLO_TRAIN_LBL = os.path.join(YOLO_DIR, 'train', 'labels')
YOLO_VALID_IMG = os.path.join(YOLO_DIR, 'valid', 'images')
YOLO_VALID_LBL = os.path.join(YOLO_DIR, 'valid', 'labels')

# Параметри генерації
SCENE_SIZE = (640, 640) # Стандартний розмір для YOLO
FONT_SIZES = [40, 50, 60, 70, 80]
MAX_DIGITS_IN_NUMBER = 5 # Генеруватимемо числа від 1 до 5 цифр

NUM_TRAIN_IMAGES = 5000 # Кількість тренувальних зображень
NUM_VALID_IMAGES = 1000 # Кількість валідаційних (тестових) зображень

# Очищення/створення директорій
if os.path.exists(YOLO_DIR):
    print(f"Очищую стару директорію: {YOLO_DIR}...")
    shutil.rmtree(YOLO_DIR)

for path in [YOLO_TRAIN_IMG, YOLO_TRAIN_LBL, YOLO_VALID_IMG, YOLO_VALID_LBL]:
    os.makedirs(path, exist_ok=True)

print(f"Структуру директорій створено в {YOLO_DIR}")

--- Етап 1: Конфігурація YOLO ---
Очищую стару директорію: learning/data_number...
Структуру директорій створено в learning/data_number


In [11]:
print("--- Етап 2: Налаштування генераторів ---")

def get_font_files(directory):
    try:
        files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.ttf')]
        if not files:
            print(f"Попередження: Директорія {directory} порожня.")
        return files
    except FileNotFoundError:
        print(f"Помилка: Директорія {directory} не знайдена.")
        return []

train_fonts = get_font_files(FONT_DIR)
valid_fonts = get_font_files(VALID_FONT_DIR)

if not train_fonts or not valid_fonts:
    print("ПОМИЛКА: Не знайдено шрифтів. Зупинка.")
else:
    print("Шрифти знайдені")

--- Етап 2: Налаштування генераторів ---
Шрифти знайдені


In [13]:
def create_number_block(number_str, font_path, font_size):
    """
    Створює "блок" з цифр на прозорому фоні і повертає
    зображення та bounding boxes [class, x_min, y_min, x_max, y_max]
    """
    try:
        font = ImageFont.truetype(font_path, font_size)
    except IOError:
        font = ImageFont.load_default()

    bboxes = []
    images = []
    current_x = 0

    for char in number_str:
        bbox = font.getbbox(char)
        char_width = bbox[2] - bbox[0]
        char_height = bbox[3] - bbox[1]

        char_img = Image.new('L', (char_width + 4, font_size + 10), 0) # 'L' = grayscale
        draw = ImageDraw.Draw(char_img)
        draw.text((2 - bbox[0], 5 - bbox[1]), char, font=font, fill=255)

        x_min = current_x + 2
        y_min = 5
        x_max = x_min + char_width
        y_max = y_min + char_height

        bboxes.append([int(char), x_min, y_min, x_max, y_max]) # [class, x1, y1, x2, y2]

        images.append(char_img)
        # Додаємо невеликий випадковий "кернінг"
        current_x += char_img.width + random.randint(-5, 2)

    total_width = current_x
    block_height = font_size + 10
    if total_width <= 0 or block_height <= 0:
        return None, None # Не вдалося створити блок

    block_img = Image.new('L', (total_width, block_height), 0)

    current_x = 0
    for img in images:
        block_img.paste(img, (current_x, 0))
        current_x += img.width + random.randint(-5, 2)

    return np.array(block_img), bboxes

# 1. Аугментації "Блоку" (трансформації "як одне ціле")
block_transform = A.Compose([
    A.Perspective(scale=(0.02, 0.08), pad_mode=0, p=0.7),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=(-0.2, 0.2), rotate_limit=10, p=0.8),
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'])) # pascal_voc = [x_min, y_min, x_max, y_max]

# 2. Аугментації "Сцени" (шум, ефекти "старого ТБ")
scene_transform = A.Compose([
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
    A.MotionBlur(blur_limit=(3, 10), p=0.4),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.6),
    A.Blur(blur_limit=(3, 5), p=0.3),
])

print("Пайплайни аугментацій налаштовано.")

Пайплайни аугментацій налаштовано.


/tmp/ipykernel_3643/848281344.py:51: UserWarning: Argument(s) 'pad_mode' are not valid for transform Perspective
  A.Perspective(scale=(0.02, 0.08), pad_mode=0, p=0.7),
/home/yesman/tfvenv/lib/python3.12/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipykernel_3643/848281344.py:57: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
/home/yesman/tfvenv/lib/python3.12/site-packages/albumentations/augmentations/blur/functional.py:232: UserWarning: blur_limit: Non-zero kernel sizes must be odd. Range (3, 10) automatically adjusted to (3, 11).
  result = _ensure_odd_values(result, info.field_name)


In [14]:
def create_yolo_sample(img_save_path, lbl_save_path, fonts):
    """
    Генерує один зразок: зображення та .txt файл з анотацією
    """

    # 1. Створюємо фон "сцени" (з легким шумом)
    scene_bg = np.random.randint(0, 40, SCENE_SIZE, dtype=np.uint8)

    # 2. Генеруємо "блок" з цифрами
    number_len = random.randint(1, MAX_DIGITS_IN_NUMBER)
    number_str = "".join([str(random.randint(0, 9)) for _ in range(number_len)])
    font_path = random.choice(fonts)
    font_size = random.choice(FONT_SIZES)

    block_img, bboxes = create_number_block(number_str, font_path, font_size)
    if block_img is None:
        return False # Не вдалося згенерувати блок

    block_h, block_w = block_img.shape
    if block_h <= 0 or block_w <= 0:
        return False

    # 3. Трансформуємо "блок" (Рівень 1)
    class_labels = [b[0] for b in bboxes]
    pascal_bboxes = [b[1:] for b in bboxes]

    try:
        transformed = block_transform(image=block_img, bboxes=pascal_bboxes, class_labels=class_labels)
        transformed_img = transformed['image']
        transformed_bboxes = transformed['bboxes']
        transformed_labels = transformed['class_labels']
    except Exception as e:
        # Іноді bbox може вийти за межі при сильному повороті. Пропустимо.
        # print(f"Помилка аугментації блоку: {e}")
        return False

    t_h, t_w = transformed_img.shape

    # 4. Вклеюємо "блок" на "сцену"
    max_x = SCENE_SIZE[0] - t_w
    max_y = SCENE_SIZE[1] - t_h
    if max_x <= 0 or max_y <= 0:
        # print("Блок завеликий для сцени, пропуск.")
        return False # Блок завеликий, пропустимо

    paste_x = random.randint(0, max_x)
    paste_y = random.randint(0, max_y)

    # Створюємо маску для вставки (щоб вставити тільки білі пікселі)
    mask = (transformed_img > 0).astype(np.uint8) * 255
    # Використовуємо 'where' для коректної вставки не-прямокутного об'єкта
    scene_bg[paste_y:paste_y+t_h, paste_x:paste_x+t_w] = np.where(
        mask > 0,
        transformed_img,
        scene_bg[paste_y:paste_y+t_h, paste_x:paste_x+t_w]
    )

    # 5. Трансформуємо "сцену" (Рівень 2)
    final_scene_img = scene_transform(image=scene_bg)['image']

    # 6. Готуємо .txt файл
    yolo_labels = []
    for label, bbox in zip(transformed_labels, transformed_bboxes):
        # Bbox - це [x_min, y_min, x_max, y_max] *відносно блоку*

        # Перераховуємо координати з відносних (0-1) у абсолютні (пікселі сцени)
        abs_x_min = bbox[0] + paste_x
        abs_y_min = bbox[1] + paste_y
        abs_x_max = bbox[2] + paste_x
        abs_y_max = bbox[3] + paste_y

        # Конвертуємо у YOLO формат (class_id, x_center_norm, y_center_norm, width_norm, height_norm)
        x_center = (abs_x_min + abs_x_max) / 2 / SCENE_SIZE[0]
        y_center = (abs_y_min + abs_y_max) / 2 / SCENE_SIZE[1]
        width = (abs_x_max - abs_x_min) / SCENE_SIZE[0]
        height = (abs_y_max - abs_y_min) / SCENE_SIZE[1]

        # Перевірка, чи bbox не виліз за межі (YOLO не любить це)
        if width > 0 and height > 0 and 0.0 < x_center < 1.0 and 0.0 < y_center < 1.0:
             yolo_labels.append(f"{label} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    if not yolo_labels:
        # print("Немає валідних bbox'ів після аугментації, пропуск.")
        return False # Немає валідних bbox'ів

    # 7. Зберігаємо
    Image.fromarray(final_scene_img).save(img_save_path)
    with open(lbl_save_path, 'w') as f:
        f.write("\n".join(yolo_labels))

    return True

# ---------------------------------
# 5. ГОЛОВНИЙ ЦИКЛ ГЕНЕРАЦІЇ
# ---------------------------------
print("--- Етап 3: Генерація TRAIN сету ---")
generated_count = 0
with tqdm(total=NUM_TRAIN_IMAGES, desc="Генерація TRAIN") as pbar:
    while generated_count < NUM_TRAIN_IMAGES:
        img_path = os.path.join(YOLO_TRAIN_IMG, f"img_{generated_count:05d}.png")
        lbl_path = os.path.join(YOLO_TRAIN_LBL, f"img_{generated_count:05d}.txt")
        if create_yolo_sample(img_path, lbl_path, train_fonts):
            generated_count += 1
            pbar.update(1)

print(f"Згенеровано {NUM_TRAIN_IMAGES} TRAIN зразків.")

--- Етап 3: Генерація TRAIN сету ---


Генерація TRAIN: 100%|██████████████████████████████████████████████████████████████| 5000/5000 [05:02<00:00, 16.55it/s]

Згенеровано 5000 TRAIN зразків.


In [15]:
print("--- Етап 4: Генерація VALID сету ---")
generated_count = 0
with tqdm(total=NUM_VALID_IMAGES, desc="Генерація VALID") as pbar:
    while generated_count < NUM_VALID_IMAGES:
        img_path = os.path.join(YOLO_VALID_IMG, f"img_{generated_count:05d}.png")
        lbl_path = os.path.join(YOLO_VALID_LBL, f"img_{generated_count:05d}.txt")
        if create_yolo_sample(img_path, lbl_path, valid_fonts):
            generated_count += 1
            pbar.update(1)

print(f"Згенеровано {NUM_VALID_IMAGES} VALID зразків.")
print(f"--- ✅ Генерацію YOLO датасету завершено! Створено в {YOLO_DIR} ---")

--- Етап 4: Генерація VALID сету ---


Генерація VALID: 100%|██████████████████████████████████████████████████████████████| 1000/1000 [00:58<00:00, 17.05it/s]

Згенеровано 1000 VALID зразків.
--- ✅ Генерацію YOLO датасету завершено! Створено в learning/data_number ---
